# Welcome to Exkaldi

In this section, we will decode based on HMM-GMM model and WFST.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Prepare a word-id table. We use the lexicons generated in early step directly. So load it. 

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

In lexicons, call "words" to get the word-id table if you want decode in words level. Or call "phones" to get the phone-ID table when decoding in phone level. But both them will return the exkaldi __ListTable__ object.

Then prepare the acoustic feature for test. We compute the feature as same as training data.

In [ ]:
scpFile = os.path.join(dataDir, "test", "wav.scp")
utt2spkFile = os.path.join(dataDir, "test", "utt2spk")
spk2uttFile = os.path.join(dataDir, "test", "spk2utt")

feat = exkaldi.compute_mfcc(scpFile, name="mfcc")
cmvn = exkaldi.compute_cmvn_stats(feat, spk2utt=spk2uttFile, name="cmvn")
feat = exkaldi.use_cmvn(feat, cmvn, utt2spk=utt2spkFile)

feat.dim

Save it to file.

In [ ]:
featFile = os.path.join(dataDir, "exp", "test_mfcc_cmvn.ark")

feat.save(featFile)

In [ ]:
feat = feat.add_delta(order=2)

feat.dim

Prepare the HMM-GMM model and WFST decoding graph. They have been generated in early steps.

In [ ]:
HCLGFile = os.path.join(dataDir, "exp", "train_delta", "graph", "HCLG.fst")

hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")

Then decode. You can set some decoding parameters such as __beam__, __acwt__ and so on. Here we only use default configures.

In [ ]:
lat = exkaldi.decode.wfst.gmm_decode(feat, hmmFile, HCLGFile, symbolTable=lexicons("words"))

lat

___lat___ is an exkaldi __Lattice__ object. We will introduce it's property in detail in next step. Now, save it to file with kaldi format.

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_delta", "decode_test")

exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)

In [ ]:
latFile = os.path.join(outDir,"test.lat")

lat.save(latFile)